In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect
"""

NEMO_DIR_PATH = "NeMo"
BRANCH = 'main'

! git clone https://github.com/NVIDIA/NeMo
%cd NeMo
! python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
%cd ..

In [ ]:
# Install dependencies
!apt-get install sox libsndfile1 ffmpeg
!pip install wget
!pip install unidecode
!pip install "matplotlib>=3.3.2"

# Multispeaker Simulator

This tutorial shows how to use the speech data simulator to generate synthetic multispeaker audio sessions that can be used to train or evaluate models for multispeaker ASR or speaker diarization. This tool aims to address the lack of labelled multispeaker training data and to help models deal with overlapping speech.

# Step 1: Download Required Datasets

The LibriSpeech dataset and corresponding word alignments are required for generating synthetic multispeaker audio sessions. For simplicity, only the dev-clean dataset is used for generating synthetic sessions.

In [3]:
import os

# download scripts if not already there 
if not os.path.exists('NeMo/scripts'):
  print("Downloading necessary scripts")
  !mkdir -p NeMo/scripts/dataset_processing
  !mkdir -p NeMo/scripts/speaker_tasks
  !wget -P NeMo/scripts/dataset_processing/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/dataset_processing/get_librispeech_data.py
  !wget -P NeMo/scripts/speaker_tasks/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/speaker_tasks/create_alignment_manifest.py

--2022-09-13 13:03:05--  https://raw.githubusercontent.com/NVIDIA/NeMo//scripts/dataset_processing/get_librispeech_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /NVIDIA/NeMo/scripts/dataset_processing/get_librispeech_data.py [following]
--2022-09-13 13:03:05--  https://raw.githubusercontent.com/NVIDIA/NeMo/scripts/dataset_processing/get_librispeech_data.py
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 404 Not Found
2022-09-13 13:03:05 ERROR 404: Not Found.

--2022-09-13 13:03:05--  https://raw.githubusercontent.com/NVIDIA/NeMo//scripts/speaker_tasks/create_alignment_manifest.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133,

In [4]:
!mkdir -p LibriSpeech
!python {NEMO_DIR_PATH}/scripts/dataset_processing/get_librispeech_data.py \
  --data_root LibriSpeech \
  --data_sets dev_clean

python: can't open file '/home/taejinp/projects/msdd_scripts_docs/NeMo/tutorials/tools/{NEMO_DIR_PATH}/scripts/dataset_processing/get_librispeech_data.py': [Errno 2] No such file or directory


The LibriSpeech forced word alignments are from [this repository.](https://github.com/CorentinJ/librispeech-alignments). You can access to the whole LibriSpeech splits at this google drive link [link](https://drive.google.com/file/d/1WYfgr31T-PPwMcxuAq09XZfHQO5Mw8fE/view?usp=sharing). We will download the dev-clean part for demo purpose.

In [5]:
!wget https://dldata-public.s3.us-east-2.amazonaws.com/LibriSpeech_Alignments.tar.gz
!tar -xvzf LibriSpeech_Alignments.tar.gz
!rm LibriSpeech-Alignments.zip

--2022-09-13 13:03:15--  https://dldata-public.s3.us-east-2.amazonaws.com/LibriSpeech_Alignments.tar.gz
Resolving dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)... 52.219.110.42
Connecting to dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)|52.219.110.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 284061 (277K) [application/x-gzip]
Saving to: ‘LibriSpeech_Alignments.tar.gz.1’

LibriSpeech_Alignme 100%[===================>] 277.40K   960KB/s    in 0.3s    

2022-09-13 13:03:15 (960 KB/s) - ‘LibriSpeech_Alignments.tar.gz.1’ saved [284061/284061]

LibriSpeech_Alignments/
LibriSpeech_Alignments/LibriSpeech/
LibriSpeech_Alignments/LibriSpeech/dev-clean/
LibriSpeech_Alignments/LibriSpeech/dev-clean/7850/
LibriSpeech_Alignments/LibriSpeech/dev-clean/7850/111771/
LibriSpeech_Alignments/LibriSpeech/dev-clean/7850/111771/7850-111771.alignment.txt
LibriSpeech_Alignments/LibriSpeech/dev-clean/7850

rm: cannot remove 'LibriSpeech-Alignments.zip': No such file or directory


# Step 2: Produce Manifest File with Forced Alignments

The LibriSpeech manifest file and LibriSpeech forced alignments will now be merged into one manifest file for ease of use when generating synthetic data. 

Here, the input LibriSpeech alignments are first converted to CTM files, and the CTM files are then combined with the base manifest in order to produce the manifest file with word alignments. When using another dataset, the --use_ctm argument can be used to generate the manifest file using alignments in CTM files.

In [2]:
!python NeMo/scripts/speaker_tasks/create_alignment_manifest.py \
  --input_manifest_filepath LibriSpeech/dev_clean.json \
  --base_alignment_path LibriSpeech_Alignments \
  --output_manifest_filepath ./dev-clean-align.json \
  --ctm_directory ./ctm_out \
  --dataset dev-clean

python: can't open file '/home/taejinp/projects/msdd_scripts_docs/NeMo/tutorials/tools/NeMo/scripts/speaker_tasks/create_alignment_manifest.py': [Errno 2] No such file or directory


# Step 3: Download Background Noise Data

The background noise is constructed from [here](https://www.openslr.org/resources/28/rirs_noises.zip) (although it can be constructed from other background noise datasets instead)

In [ ]:
!wget https://www.openslr.org/resources/28/rirs_noises.zip
!unzip -o rirs_noises.zip

# Data Simulator Overview

The simulator creates a speech session using utterances from a desired number of speakers. The simulator first selects the LibriSpeech speaker IDs that will be used for the current session, and sets speaker dominance values for each speaker to determine how often each speaker will talk in the session. The session is then constructed by iterating through the following steps:

* The next speaker is selected (which could be the same speaker again with some probability, and which accounts for the speaker dominance values).
* The sentence length is determined using a probability distribution, and an utterance of the desired length is then constructed by concatenating together (or truncating) LibriSpeech sentences corresponding to the desired speaker. Individual word alignments are used to truncate the last LibriSpeech sentence such that the entire utterance has the desired length.
* Next, either the current utterance is overlapped with a previous utterance or silence is introduced before inserting the current utterance. 

The simulator includes a multi-microphone far-field mode that incorporates synthetic room impulse response generation in order to simulate multi-microphone multispeaker sessions. When using RIR generation, the RIR is computed once per batch of sessions, and then each constructed utterance is convolved with the RIR in order to get the sound recorded by each microphone before adding the utterance to the audio session. In this tutorial, only near field sessions will be generated.

The simulator also has a speaker enforcement mode which ensures that the correct number of speakers appear in each session, since it is possible that fewer than the desired number may be present since speaker turns are probabilistic. In speaker enforcement mode, the length of the session or speaker probabilites may be adjusted to ensure all speakers are included before the session finishes.

# Parameters

In [ ]:
from omegaconf import OmegaConf
ROOT = os.getcwd()
conf_dir = os.path.join(ROOT,'conf')
!mkdir -p $conf_dir
CONFIG_PATH = os.path.join(conf_dir, 'data_simulator.yaml')
if not os.path.exists(CONFIG_PATH):
  !wget -P $conf_dir https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/tools/speech_data_simulator/conf/data_simulator.yaml

config = OmegaConf.load(CONFIG_PATH)
print(OmegaConf.to_yaml(config))

# Step 4: Create Background Noise Manifest

In [ ]:
!find RIRS_NOISES/real_rirs_isotropic_noises/*.wav > bg_noise.list 

# this function can also be run using the pathfiles_to_diarize_manifest.py script
from nemo.collections.asr.parts.utils.manifest_utils import create_manifest
create_manifest('bg_noise.list', 'bg_noise.json')

# Step 5: Generate Simulated Audio Session

A single 4-speaker session of 60 seconds is generated as an example. 

In [ ]:
from nemo.collections.asr.data.data_simulation import MultiSpeakerSimulator

ROOT = os.getcwd()
data_dir = os.path.join(ROOT,'simulated_data')
config.data_simulator.random_seed=42
config.data_simulator.manifest_filepath="./dev-clean-align.json"
config.data_simulator.outputs.output_dir=data_dir
config.data_simulator.session_config.num_sessions=1
config.data_simulator.session_config.session_length=30
config.data_simulator.background_noise.add_bg=True
config.data_simulator.background_noise.background_manifest="./bg_noise.json"

lg = MultiSpeakerSimulator(cfg=config)
lg.generate_sessions()

# Step 5: Listen to and Visualize Session

Listen to the audio and visualize the corresponding speaker timestamps (recorded in a RTTM file for each session)

In [ ]:
import os
import wget
import IPython
import matplotlib.pyplot as plt
import numpy as np
import librosa
from nemo.collections.asr.parts.utils.speaker_utils import rttm_to_labels, labels_to_pyannote_object

ROOT = os.getcwd()
data_dir = os.path.join(ROOT,'simulated_data')
audio = os.path.join(data_dir,'multispeaker_session_0.wav')
rttm = os.path.join(data_dir,'multispeaker_session_0.rttm')

sr = 16000
signal, sr = librosa.load(audio,sr=sr) 

fig,ax = plt.subplots(1,1)
fig.set_figwidth(20)
fig.set_figheight(2)
plt.plot(np.arange(len(signal)),signal,'gray')
fig.suptitle('Synthetic Audio Session', fontsize=16)
plt.xlabel('Time (s)', fontsize=18)
ax.margins(x=0)
plt.ylabel('Signal Strength', fontsize=16);
a,_ = plt.xticks();plt.xticks(a,a/sr);

IPython.display.Audio(audio)

The visualization is useful for seeing both the distribution of utterance lengths, the differing speaker dominance values, and the amount of overlap in the session.

In [ ]:
#display speaker labels for reference
labels = rttm_to_labels(rttm)
reference = labels_to_pyannote_object(labels)
reference

# Files Produced

The following files are produced by the simulator:

* wav files (one per audio session) - the output audio sessions
* rttm files (one per audio session) - the speaker timestamps for the corresponding audio session (used for diarization training)
* json files (one per audio session) - the output manifest file for the corresponding audio session (containing text transcriptions, utterance durations, full paths to audio files, words, and word alignments)
* ctm files (one per audio session) - contains word-by-word alignments, speaker ID, and word
* txt files (one per audio session) - contains the full text transcription for a given session
* list files (one per file type per batch of sessions) - a list of generated files of the given type (wav, rttm, json, ctm, or txt), used primarily for manifest creation


In [ ]:
!ls simulated_data